In [7]:
import pandas as pd
import os
import glob
import CBE_utils as CBE

In [8]:
raw_dir = "/media/schmied.christopher/T7 Shield/Datasets/ECBL/raw/"
processed_dir = "/media/schmied.christopher/T7 Shield/Datasets/ECBL/processed/"

annotation_dir = "/home/schmied.christopher/FMP_Docs/Projects/eu_os_ecbl_qc/annotation/"

In [9]:
def list_folders(directory):
    try:
        # List all entries in the specified directory
        entries = os.listdir(directory)
        
        # Filter out non-folder entries
        folders = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
        return folders
    
    except FileNotFoundError:
        
        return f"The directory '{directory}' does not exist."
    
    except PermissionError:
        
        return f"Permission denied to access the directory '{directory}'."

In [47]:
# List of all files in raw data folder

raw_pattern = "*_CP_Profiles_Aggregated.csv"

raw_source_list = list_folders(raw_dir)

raw_files = []

for raw_source in raw_source_list:

    print(raw_source)

    raw_plate_list = list_folders(os.path.join(raw_dir, raw_source))

    for raw_plate in raw_plate_list:

        print(raw_plate)

        raw_batch_list = list_folders(os.path.join(raw_dir, raw_source, raw_plate))

        for raw_batch in raw_batch_list:

            print(raw_batch)

            if glob.glob(os.path.join(raw_dir, raw_source, raw_plate, raw_batch) + os.sep + raw_pattern , recursive=True):
                
                raw_file_exists = True
            
            else:
                
                raw_file_exists = False

            print(raw_file_exists)


            raw_files.append({'source': raw_source,
                              'plate': raw_plate,
                              'batch':raw_batch,
                              'raw_file': raw_file_exists})

FMP
C1084R1
240605
True
250813
True
C1084R2
240614
True
250815
True
C1084R3
250827
True
C1084R4
240621
True
251210
True
C1085R1
240605
True
250813
True
C1085R2
240614
True
250815
True
C1085R3
250827
True
C1085R4
240621
True
250829
True
C1086R1
240605
True
250813
True
C1086R2
240614
True
250815
True
C1086R3
250827
True
C1086R4
240621
True
250829
True
C1087R1
240605
True
250813
True
C1087R2
240614
True
250815
True
C1087R3
250827
True
C1087R4
240621
True
250829
True
C1088R1
240605
True
250813
True
C1088R2
240614
True
250815
True
C1088R3
250827
True
C1088R4
240621
True
250829
True
C1089R1
240605
True
250813
True
C1089R2
240614
True
250815
True
C1089R3
250827
True
C1089R4
240621
True
250829
True
C1090R1
240505
True
250813
True
C1090R2
240614
True
250815
True
C1090R3
250827
True
C1090R4
240621
True
250829
True
C1091R1
240605
True
250813
True
C1091R2
240614
True
250815
True
C1091R3
250827
True
C1091R4
240621
True
250829
True
C1092R1
240605
True
250813
True
C1092R2
240614
True
250815
True
C109

In [ ]:
# Turn plate into plate_name and replicate_number C1084 R1
# Create plate_map_name C1084_R1
raw_files_df = pd.DataFrame(raw_files)
raw_files_df["plate_name"] = raw_files_df["plate"].str[:5]
raw_files_df["replicate_number"] = raw_files_df["plate"].str[5:]
raw_files_df["plate_map_name"] = raw_files_df["plate_name"].str.cat(raw_files_df["replicate_number"], sep="_")

In [63]:
# Turn batch into batch_date, protect against 000000 batch 
raw_files_df["batch_date"] = pd.to_datetime(
    raw_files_df["batch"].replace("000000", pd.NA),
    format="%y%m%d",
    errors="coerce"
)

# Create a batch_date_str column for easier comparison with plate map batch dates, fill NA with 0000-00-00
raw_files_df["batch_date_str"] = (
    raw_files_df["batch_date"]
    .dt.strftime("%Y-%m-%d")
    .fillna("0000-00-00")
)

In [64]:
raw_column = raw_files_df.pop("raw_file")
raw_files_df["raw_file"] = raw_column

In [65]:
raw_files_df

,source,plate,batch,plate_name,replicate_number,plate_map_name,batch_date,batch_date_str,raw_file
0,FMP,C1084R1,240605,C1084,R1,C1084_R1,2024-06-05,2024-06-05,True
1,FMP,C1084R1,250813,C1084,R1,C1084_R1,2025-08-13,2025-08-13,True
2,FMP,C1084R2,240614,C1084,R2,C1084_R2,2024-06-14,2024-06-14,True
3,FMP,C1084R2,250815,C1084,R2,C1084_R2,2025-08-15,2025-08-15,True
4,FMP,C1084R3,250827,C1084,R3,C1084_R3,2025-08-27,2025-08-27,True
...,...,...,...,...,...,...,...,...,...
1441,USC,USC02R1,240726,USC02,R1,USC02_R1,2024-07-26,2024-07-26,True
1442,USC,USC03R1,240927,USC03,R1,USC03_R1,2024-09-27,2024-09-27,True
1443,USC,USC04R1,241004,USC04,R1,USC04_R1,2024-10-04,2024-10-04,True
1444,USC,USC05R1,241016,USC05,R1,USC05_R1,2024-10-16,2024-10-16,True


In [ ]:
# Based on raw data check if these files have processed counterparts in the processed folder
# Walk through source
# Get batch_date, plate_map_name and check if file exists in processed folder

for source in raw_files_df["source"].unique():

    print(source)

    processed_source_path = os.path.join(processed_dir, source)

    raw_files_df_source = raw_files_df[raw_files_df["source"] == source]

    for index, row in raw_files_df_source.iterrows():
        
        plate_folder = row["batch_date_str"] + "_" + row["plate_map_name"]
        
        print(plate_folder)
        
        plate_path = os.path.join(processed_source_path, plate_folder)

        if os.path.isdir(plate_path):

            process_folder = True
            
        else:
            
            process_folder = False

        processed_pattern = "[A-Z][A-Za-z0-9][A-Za-z0-9][0-9][0-9]_R[1-4].csv"

        if glob.glob(plate_path + os.sep + processed_pattern, recursive=True):

            processed_file_exists = True

        else:

            processed_file_exists = False

        norm_pattern = "[A-Z][A-Za-z0-9][A-Za-z0-9][0-9][0-9]_R[1-4]_mad_robustize.csv"

        if glob.glob(plate_path + os.sep + norm_pattern  , recursive=True):

            norm_file_exits = True
            
        else:

            norm_file_exits = False
       
        processed_pattern = "[A-Z][A-Za-z0-9][A-Za-z0-9][0-9][0-9]_R[1-4]_mad_robustize_reduced-corr.csv"
        
        if glob.glob(plate_path + os.sep + processed_pattern , recursive=True):

            reduced_file_exists = True
            
        else:

            reduced_file_exists = False

        raw_files_df.at[index, "process_folder"] = process_folder
        raw_files_df.at[index, "processed_file_exists"] = processed_file_exists
        raw_files_df.at[index, "norm_file_exists"] = norm_file_exits
        raw_files_df.at[index, "reduced_file_exists"] = reduced_file_exists
        

FMP
2024-06-05_C1084_R1
2025-08-13_C1084_R1
2024-06-14_C1084_R2
2025-08-15_C1084_R2
2025-08-27_C1084_R3
2024-06-21_C1084_R4
2025-12-10_C1084_R4
2024-06-05_C1085_R1
2025-08-13_C1085_R1
2024-06-14_C1085_R2
2025-08-15_C1085_R2
2025-08-27_C1085_R3
2024-06-21_C1085_R4
2025-08-29_C1085_R4
2024-06-05_C1086_R1
2025-08-13_C1086_R1
2024-06-14_C1086_R2
2025-08-15_C1086_R2
2025-08-27_C1086_R3
2024-06-21_C1086_R4
2025-08-29_C1086_R4
2024-06-05_C1087_R1
2025-08-13_C1087_R1
2024-06-14_C1087_R2
2025-08-15_C1087_R2
2025-08-27_C1087_R3
2024-06-21_C1087_R4
2025-08-29_C1087_R4
2024-06-05_C1088_R1
2025-08-13_C1088_R1
2024-06-14_C1088_R2
2025-08-15_C1088_R2
2025-08-27_C1088_R3
2024-06-21_C1088_R4
2025-08-29_C1088_R4
2024-06-05_C1089_R1
2025-08-13_C1089_R1
2024-06-14_C1089_R2
2025-08-15_C1089_R2
2025-08-27_C1089_R3
2024-06-21_C1089_R4
2025-08-29_C1089_R4
2024-05-05_C1090_R1
2025-08-13_C1090_R1
2024-06-14_C1090_R2
2025-08-15_C1090_R2
2025-08-27_C1090_R3
2024-06-21_C1090_R4
2025-08-29_C1090_R4
2024-06-05_C1091

In [75]:
raw_files_df

,source,plate,batch,plate_name,replicate_number,plate_map_name,batch_date,batch_date_str,raw_file,process_folder,processed_file_exists,norm_file_exists,reduced_file_exists
0,FMP,C1084R1,240605,C1084,R1,C1084_R1,2024-06-05,2024-06-05,True,False,False,False,False
1,FMP,C1084R1,250813,C1084,R1,C1084_R1,2025-08-13,2025-08-13,True,True,True,True,True
2,FMP,C1084R2,240614,C1084,R2,C1084_R2,2024-06-14,2024-06-14,True,False,False,False,False
3,FMP,C1084R2,250815,C1084,R2,C1084_R2,2025-08-15,2025-08-15,True,True,True,True,True
4,FMP,C1084R3,250827,C1084,R3,C1084_R3,2025-08-27,2025-08-27,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,USC,USC02R1,240726,USC02,R1,USC02_R1,2024-07-26,2024-07-26,True,True,True,True,False
1442,USC,USC03R1,240927,USC03,R1,USC03_R1,2024-09-27,2024-09-27,True,True,True,True,False
1443,USC,USC04R1,241004,USC04,R1,USC04_R1,2024-10-04,2024-10-04,True,True,True,True,False
1444,USC,USC05R1,241016,USC05,R1,USC05_R1,2024-10-16,2024-10-16,True,True,True,True,False
